# Classificação de texto

Neste exemplo, utilizaremos um dataset chamado 20newsgroups. 

O conjunto de dados é uma coleção de aproximadamente 20.000 documentos de grupos de notícias, particionados (quase) uniformemente em 20 grupos/categorias de notícias diferentes. Mais informações sobre esta base podem ser obtidas no repositório [UCI](http://archive.ics.uci.edu/ml/datasets/Twenty+Newsgroups)

As categorias existentes são:

* 'alt.atheism',
* 'comp.graphics',
* 'comp.os.ms-windows.misc',
* 'comp.sys.ibm.pc.hardware',
* 'comp.sys.mac.hardware',
* 'comp.windows.x',
* 'misc.forsale',
* 'rec.autos',
* 'rec.motorcycles',
* 'rec.sport.baseball',
* 'rec.sport.hockey',
* 'sci.crypt',
* 'sci.electronics',
* 'sci.med',
* 'sci.space',
* 'soc.religion.christian',
* 'talk.politics.guns',
* 'talk.politics.mideast',
* 'talk.politics.misc',
* 'talk.religion.misc'

Neste exemplo, vamos considerar apenas duas categorias: '**alt.atheism**' e '**comp.graphics**'. O scitkit contém uma função que auxilia o download desta base:

In [ ]:
from sklearn.datasets import fetch_20newsgroups
categories = [
    'alt.atheism',
    'comp.graphics',
]

twenty = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'), shuffle=True)

# **Visualizar descrição**

In [ ]:
print(twenty.DESCR)

In [ ]:
print(twenty.target_names)

In [ ]:
print(twenty.data[0:3])

In [ ]:
print(twenty.target[0:3])

# **Converter para pandas**

In [ ]:
import pandas as pd
import numpy as np

df = pd.DataFrame([twenty.data, twenty.target.tolist()]).T
df.columns = ['text', 'target']
targets = pd.DataFrame( twenty.target_names)
targets.columns=['title']
data = pd.merge(df, targets, left_on='target', right_index=True)

data.head()

In [ ]:
data.describe()

## Pré-processamento e Análise exploratória

## Número de documentos por classe:

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


fig = plt.figure(figsize=(8,4))
sns.barplot(x = data['target'].unique(), y=data['target'].value_counts())
plt.show()

## Usando o spacy para pré-processamento:

In [ ]:
import spacy
nlp = spacy.load('en')

In [ ]:
def limpeza(documentos):
  textos = []
  for doc in documentos:
    doc = doc.replace("\n", " ").replace("\r", " ")
    doc_processado = nlp(doc)
    tokens = [token.lemma_.lower() for token in doc_processado if (not token.is_stop) 
              and (not token.is_punct) and (token.lemma_ != '-PRON-') and 
              (len(token.lemma_)>3) and (token.lemma_ != '\n') and (token.lemma_ != ' ')]
    tokens = ' '.join(tokens)
    textos.append(tokens)
  return pd.Series(textos)

documentos = [doc for doc in data['text']]

documentos_processados = limpeza(documentos)

documentos_processados

## Contagem de número de palavras do banco de dados

In [ ]:
from collections import Counter

todos_documentos = ' '.join(documentos_processados).split()


contagem_de_palavras = Counter(todos_documentos)

palavras_comuns = [word[0] for word in contagem_de_palavras.most_common(20)]
palavras_comuns_contagem = [word[1] for word in contagem_de_palavras.most_common(20)]

fig = plt.figure(figsize=(18,6))
sns.barplot(x=palavras_comuns, y=palavras_comuns_contagem)
plt.title('Top-20 palavras mais comuns')
plt.show()

## Atualizar o pandas

In [ ]:
data['text'] = documentos_processados.values
data

## Separar treinamento e teste

In [ ]:
from sklearn.model_selection import train_test_split

df_train, df_test = train_test_split(data, test_size=0.3)
print('Número de exemplos de treinamento:', len(df_train))
print('Número de exemplos de teste:', len(df_test))

## Representação vetorial do texto

Neste exemplo, utilizaremos o algoritmo Naive Bayes para classificar documentos de texto em categorias. Para isso, precisamos antes converter o texto para uma representação vetorial, ou seja, cada documento/exemplo precisa ser representado por um vetor de dimensões pré-definidas. Utilizaremos três técnicas básicas: BOW (*Bag of Words*), TF (*Term Frequency*) e TF-IDF (*Term Frequency - Inverse Document Frequency*)

### BOW

Consiste basicamente em contar quantas vezes cada palavra aparece no documento. Ou seja, sua aplicação a um conjunto de *n* documentos, produz uma matriz *n x d*, onde *d* corresponde ao tamanho do vocabulário considerado. No Scikit, esta representação é implementada pelo [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer):

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
bow_model  = vectorizer.fit(df_train['text'])

X_bow_train = bow_model.transform(df_train['text'])
X_bow_test  = bow_model.transform(df_test['text'])

print(X_bow_train.shape,X_bow_test.shape)

In [ ]:
# matriz está armazenada em formato sparse
print(X_bow_train[0,:])

### *Term Frequency* (TF)
A contagem de ocorrências (i.e., BOW) é um bom começo, mas há um problema: documentos mais longos terão valores de contagem média mais altos do que documentos mais curtos, embora possam falar sobre os mesmos tópicos.

Para evitar essas possíveis discrepâncias, basta dividir o número de ocorrências de cada palavra em um documento pelo número total de palavras no documento:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=False)
tf_model = vectorizer.fit(df_train['text'])

X_tf_train = tf_model.transform(df_train['text'])
X_tf_test  = tf_model.transform(df_test['text'])

print(X_tf_train[0,:])

### TF-IDF
Usando o scikit, basta ativar o flag `use_idf=True`:

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(df_train['text'])

X_tfidf_train = tfidf_model.transform(df_train['text'])
X_tfidf_test  = tfidf_model.transform(df_test['text'])

print(X_tfidf_train[0,:])

#tfidf_transformer = TfidfTransformer(use_idf=True)
#X_train_TFIDF = tfidf_transformer.fit_transform(X_train_TF)
#X_train_TFIDF.shape

# **Classificação**

Como vimos no notebook de aprendizagem de máquina.

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

model = DecisionTreeClassifier()
model.fit(X_tfidf_train, df_train['title'])

predicoes = model.predict(X_tfidf_test)
acc = accuracy_score(df_test['title'], predicoes)
#print(model)
print('Taxa de acerto: ', accuracy_score(df_test['title'], predicoes))

## Classificação de um novo texto

In [ ]:
docs_new = ['OpenGL on the GPU is fast']

vectorizer = TfidfVectorizer(use_idf=True)
tfidf_model = vectorizer.fit(df_train['text'])

X_new_tfidf  = tfidf_model.transform(docs_new)

predicted = model.predict(X_new_tfidf)
for doc, category in zip(docs_new, predicted):
     print('%r => %s' % (doc, category))

# Bônus - TextBlob

TextBlob: [documentação](https://textblob.readthedocs.io/en/dev/)

## Requisitos: 

NLTK com ('punkt'), ('averaged_perceptron_tagger') e ('brown')


In [ ]:
from textblob import TextBlob
import nltk
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('brown')

In [ ]:
text = '''
The titular threat of The Blob has always struck me as the ultimate movie
monster: an insatiably hungry, amoeba-like mass able to penetrate
virtually any safeguard, capable of--as a doomed doctor chillingly
describes it--"assimilating flesh on contact.
Snide comparisons to gelatin be damned, it's a concept with the most
devastating of potential consequences, not unlike the grey goo scenario
proposed by technological theorists fearful of
artificial intelligence run rampant.
'''

blob = TextBlob(text)
      

for sentence in blob.sentences:
    print(sentence.sentiment.polarity)


blob.translate(to="pt")  